In [1]:
# Cryptanalysis - Task 2 - CipherC and Attack - v1

import numpy as np

#febc6d78039a4215
# different than in CipherThree and CipherFour !!!
def sbox(input):
    list_out = [15, 14, 11, 12, 6, 13, 7, 8, 0, 3, 9, 10, 4, 2, 1, 5]
    return list_out[input]

def inverse_sbox(input):
    list_out = [15, 14, 11, 12, 6, 13, 7, 8, 0, 3, 9, 10, 4, 2, 1, 5]
    return list_out.index(input)

def encrypt(keys, message):
    ct = message
    for i in range(len(keys) - 1):
        ct ^^= keys[i]
        ct = sbox(ct)
    ct ^^= keys[-1]
    return ct

def decrypt(keys, ct):
    message = ct
    message ^^= keys[-1]
    for i in range(len(keys) - 2, -1, -1):
        message ^^= keys[i]
        message = inverse_sbox(message)
    return message

def generate_pairs(amount):
    pairs = []
    count = 0
    for i in range(16):
        for j in range(i, 16):
            pairs.append((i, j))
            count += 1
            if amount is not None and count >= amount:
                return pairs
    return pairs

def generate_pts_cts(keys, amount):
    cts = []
    for i in range(amount):
        ct = encrypt(keys, i)
        cts.append((i, ct))
    return cts

def attack(keys, mask):
    
    #mask = 13  # ~d
    #mask = 11
    #mask = vector(GF(2), [1, 0, 1, 1])  # first bit is LSB
    pts_cts = generate_pts_cts(keys, 16)
        
    guesses = range(16)
    #table = [[0, 0]] * len(guesses)
    table = {}
    
    for key_guess in guesses:
        table[key_guess] = [0, 0]
        for (pt, ct) in pts_cts:
            y_guess = inverse_sbox(ct ^^ key_guess)
            
            a_0, a_1 = (pt & mask), (y_guess & mask)
            res = 0
            for i in range(4):
                res ^^= ((a_0 >> i) % 2) ^^ ((a_1 >> i) % 2)  # sum bits of a_0 and a_1 (mod 2)
            if res == 0:
                table[key_guess][0] += 1
            else:
                table[key_guess][1] += 1
    
    max_val = 0
    final_guesses = []
    for key in table.keys():
        values = table[key]
        #print(key, values)
        val = abs(values[0] - values[1])
        if val == max_val:
            final_guesses.append(key)
        elif val > max_val:
            final_guesses = [key]
            max_val = val
    
    #print("final guesses:", final_guesses)
    
    return table, final_guesses

def test_attack(mask):
    inside = [0] * 16
    bad = 0
    for i in range(4):
        for j in range(4):
            for k in range(4):
                for l in range(4):
                    keys = [i, j, k, l]
                    table, guesses = attack(keys, mask)
                    if l in guesses:
                        inside[len(guesses)] += 1
                    else:
                        bad += 1
    return inside, bad

mes_1 = 7
keys_1 = [1, 2, 3, 4]
ct_1 = encrypt(keys_1, mes_1)
dec_1 = decrypt(keys_1, ct_1)

#print("Enc and Dec:", mes_1, "->", ct_1, "->", dec_1)

#print(generate_pairs(10))

#print(len(range(3)))

#print(6 & 5)

"""
a_0, a_1 = 3, 7
res = 0
for i in range(4):
    #print(((a_0 << i) >> 3), ((a_1 << i) >> 3))
    #res ^^= ((a_0 << i) >> 3) ^^ ((a_1 << i) >> 3)
    print((a_0 >> i) % 2, (a_1 >> i) % 2)
    res ^^= (a_0 >> i) % 2 ^^ (a_1 >> i) % 2
print(res)
"""

#print(attack(keys_1, 13))

print(test_attack(13))

([0, 0, 48, 0, 112, 0, 32, 0, 16, 0, 0, 0, 0, 0, 0, 0], 48)
